In [1]:
from pyspark.sql import SparkSession
from graphframes import GraphFrame
import os

postgres_config = {
    "url": "jdbc:postgresql://postgres:5432/twitter",
    "driver": "org.postgresql.Driver",
    "user": "crawler",
    "password": "12345"
}

In [2]:
spark = SparkSession.builder.appName('twitter').config("spark.jars", os.getenv("SPARK_CLASSPATH", "")).config("spark.driver.memory", "7g").getOrCreate()

In [3]:
def get_users(limit: int):
    dbtable = f"(SELECT id, screen_name FROM users LIMIT {limit}) as u"
    return spark.read.format('jdbc').\
            options(dbtable=dbtable, **postgres_config).\
            load()

def get_edges(limit: int):
    dbtable = f"(SELECT user_id, follower_id FROM followers LIMIT {limit}) as u"
    return spark.read.format('jdbc').\
            options(dbtable=dbtable, **postgres_config).\
            load().withColumnRenamed("user_id", "dst").withColumnRenamed("follower_id", "src")

In [4]:
edges = get_edges(10000000)

In [5]:
users_df = edges.select("dst").distinct().withColumnRenamed("dst", "id")
followers_df = edges.select("src").distinct().withColumnRenamed("src", "id")
vertices = users_df.union(followers_df).distinct().cache()

In [10]:
vertices.show()

+------------------+
|                id|
+------------------+
|        2723884713|
|        3326711022|
|         250452420|
|         218927682|
|         419119609|
|        3506188937|
|         617684776|
|        3538362808|
|        1372608817|
|        1064144696|
|         146147034|
|782276232020955136|
|782255679616720896|
|        2210092992|
|759776466632450053|
|        1732140697|
|891165691788709888|
|857265425809162240|
|860489657984962560|
|926461104674557952|
+------------------+
only showing top 20 rows



In [35]:
edges.show()

+---------+---------+
|      dst|      src|
+---------+---------+
|726229723| 54614491|
|726229723|823763214|
|726229723|175406313|
|726229723|384683053|
|726229723|811172107|
|726229723|356425203|
|726229723|209216710|
|726229723|787348728|
|726229723|470849385|
|726229723|361001998|
|726229723|293916623|
|726229723|491051200|
|726229723|595847070|
|726229723|554900589|
|726229723|790046364|
|726229723|574067473|
|726229723|203270452|
|726229723|835940156|
|726229723|840559650|
|726229723|633824289|
+---------+---------+
only showing top 20 rows



In [6]:
g = GraphFrame(vertices, edges)

In [7]:
g.inDegrees.show()

+----------+--------+
|        id|inDegree|
+----------+--------+
|1635953330|     813|
|3313015059|     422|
| 116344383|       6|
| 458578709|      19|
|3937020861|    4518|
| 549984857|     449|
| 803016158|     634|
|4686288074|      57|
|  30742740|       8|
|  18019065|     276|
| 335050631|     502|
| 499007295|     216|
| 523389353|      26|
|  51263627|      19|
|3574869863|    1205|
| 142383459|     141|
|2432124493|      38|
|1950132858|     296|
|1024701650|   18015|
| 548300885|     337|
+----------+--------+
only showing top 20 rows



In [8]:
g.outDegrees.show()

+------------------+---------+
|                id|outDegree|
+------------------+---------+
|741450836421664770|       15|
|         256976741|        1|
|        1390797649|        1|
|         298954198|        1|
|        1199214380|        1|
|        1856146332|        1|
|         182200542|        2|
|         863880180|        1|
|        3071601897|        3|
|         878634900|        5|
|          28057986|        1|
|843310827331575813|        6|
|        4497848234|        1|
|        1697913798|        2|
|         102919755|        5|
|          54228009|        9|
|        1417854396|        1|
|        2373187454|        2|
|        2817901389|        2|
|         387364842|        3|
+------------------+---------+
only showing top 20 rows



In [9]:
results = g.pageRank(resetProbability=0.01, maxIter=20)
results.vertices.select("id", "pagerank").show()

Py4JJavaError: An error occurred while calling o76.run.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 1008, localhost, executor driver): java.lang.ArrayIndexOutOfBoundsException: -1
	at org.apache.spark.graphx.util.collection.GraphXPrimitiveKeyOpenHashMap$mcJI$sp.apply$mcJI$sp(GraphXPrimitiveKeyOpenHashMap.scala:64)
	at org.apache.spark.graphx.impl.EdgePartition.updateVertices(EdgePartition.scala:91)
	at org.apache.spark.graphx.impl.ReplicatedVertexView$$anonfun$2$$anonfun$apply$1.apply(ReplicatedVertexView.scala:73)
	at org.apache.spark.graphx.impl.ReplicatedVertexView$$anonfun$2$$anonfun$apply$1.apply(ReplicatedVertexView.scala:71)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:410)
	at org.apache.spark.graphx.impl.EdgeRDDImpl$$anonfun$mapEdgePartitions$1.apply(EdgeRDDImpl.scala:121)
	at org.apache.spark.graphx.impl.EdgeRDDImpl$$anonfun$mapEdgePartitions$1.app

In [31]:
results.vertices.select("id", "pagerank").orderBy("pagerank", ascending=False).show()

+-------------------+------------------+
|                 id|          pagerank|
+-------------------+------------------+
| 899827533310554112|53270.101178899335|
|          707438557|52995.420315590934|
|          220552737|24777.455934628244|
|          220642188|22118.906013470427|
|          221002173| 22061.40781990723|
|          114685098| 22047.74244208183|
|          202994739|20362.317343458035|
|          221231171| 18367.56526774537|
|          220351020| 17973.81695969523|
|          134371793|13618.817189786798|
|           74183106|12120.747513969774|
|1010498044624867328|11462.348377571388|
|          178665984|11246.601882149183|
|           35806081|10400.890367290876|
|          200374062| 8740.764933210634|
|           46257296|7591.8944366193045|
|          106997059|  7395.92337650512|
|          111529062| 7267.433332326625|
|          111522651| 7113.842859223085|
|          203748405| 6763.668143996587|
+-------------------+------------------+
only showing top